In [1]:
from collections import Counter
from marked_words_occ import marked_words as get_marked_words_occ
from marked_words import marked_words as get_marked_words_no_occ
from nltk.tokenize import sent_tokenize, word_tokenize

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df_10 = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_10.csv')

In [3]:
df_100 = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_100_06-24-2024, 10:30:39.csv')

In [4]:
df_100_for_loop = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_100_06-29-2024, 13:28:28.csv')
df_10_for_loop_1 = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_10_06-26-2024, 16:30:49.csv') 

df_10_for_loop = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_10_06-26-2024, 17:13:53.csv')

df_200_for_loop = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_200_06-30-2024, 12:47:15.csv')

In [5]:
def pprint(dic):
    full_list = []
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
        if word[0].lower() == 'latina':
            print('LATINA')
        full_list.append(word[0])
    return full_list

In [6]:
def compute_marked_words(df, prior=True, frac_words=1):
    dv3_mw = {}
    for race in df['race'].unique():
    #     print('\n Top words for %s \n-------' % race)
        outs = pprint(get_marked_words_no_occ(df, [race], ['race'],['a White'], prior=prior, frac_words=frac_words))
        dv3_mw[race] = outs
    temps = []
    for race in df['race'].unique():
    #     print('\n Top words for %s \n-------' % race)
        temp = pprint(get_marked_words_no_occ(df, ['a White'], ['race'],[race], prior=prior, frac_words=frac_words))
        temps.extend(temp)
    seen = Counter(temps).most_common()
    dv3_mw['a White']=[w for w, c in seen if c == 4]


    for race in df['gender'].unique():
    #     print('\n Top words for %s \n-------' % race)
        outs = pprint(get_marked_words_no_occ(df, [race], ['gender'],['M'], prior=prior, frac_words=frac_words))
        dv3_mw[race] = outs
    temps = []
    for race in df['gender'].unique():
    #     print('\n Top words for %s \n-------' % race)
        temp = pprint(get_marked_words_no_occ(df, ['M'], ['gender'],[race], prior=prior, frac_words=frac_words))
        temps.extend(temp)

    seen = Counter(temps).most_common()
    dv3_mw['M']=[w for w, c in seen if c == 2]
        
        
    # Top words for intersectional groups
    for race in df['race'].unique():
        for gen in ['M', 'N','F']:
            dv3_mw[race+gen] = pprint(get_marked_words_no_occ(df, [race, gen], ['race', 'gender'],['a White','M'], prior=prior, frac_words=frac_words))

    marked_word_similarity = dict()
    temp = list()
    for key in list(dv3_mw.keys()):
        marked_word_similarity[key] = list()
        for word in dv3_mw[key]:
            temp.append(word_tokenize(word))
    if len(temp) > 0:
        data = temp
    # print(data)
    return dv3_mw


In [20]:
def compare_marked_words_across_gens(frac_words, prior=True, test_words_enclosed=False):
    dv3_mw_10 = compute_marked_words(df_10, prior=prior, frac_words=frac_words)
    dv3_mw_100  = compute_marked_words(df_100, prior=prior, frac_words=frac_words)
    dv3_mw_100_for = compute_marked_words(df_100_for_loop, prior=prior, frac_words=frac_words)
    dv3_mw_10_for = compute_marked_words(df_10_for_loop, prior=prior, frac_words=frac_words)
    dv3_mw_200_for = compute_marked_words(df_200_for_loop, prior=prior, frac_words=frac_words)

    for key in list(dv3_mw_10.keys()):
        if key == 'b':
            continue
        if len(dv3_mw_100[key]) == 0:
            continue
        print(key)
        print(len(dv3_mw_10[key]), '| 10 |', sorted(dv3_mw_10[key]))
        print(len(dv3_mw_100[key]), '| 100 |', sorted(dv3_mw_100[key]))
        # try:
        print(len(dv3_mw_100_for[key]), '| 100 for |', sorted(dv3_mw_100_for[key]))
        print(len(dv3_mw_200_for[key]), '| 200 for |', sorted(dv3_mw_200_for[key]))
        print(len(dv3_mw_10_for[key]), '| 10 for |', sorted(dv3_mw_10_for[key]))
    
    if test_words_enclosed:
        passed = True
        for key in list(dv3_mw_10_for.keys()):
            if key == 'b':
                continue
            if len(dv3_mw_10_for[key]) == 0:
                continue
            for word in dv3_mw_10_for[key]:
                if word not in dv3_mw_100_for[key]:
                    print(f"FAILED on {word} in {key} 10 - 100")
                    passed = False
        for key in list(dv3_mw_100_for.keys()):
            if key == 'b':
                continue
            if len(dv3_mw_100[key]) == 0:
                continue
            for word in dv3_mw_100_for[key]:
                if word not in dv3_mw_200_for[key]:
                    print(f"FAILED on {word} in {key} 100 - 200")
                    passed = False
        if passed:
            print("PASSED")
    return dv3_mw_10, dv3_mw_100, dv3_mw_100_for, dv3_mw_10_for, dv3_mw_100_for

In [8]:
# no prior
x = compare_marked_words_across_gens(1, False)

a White
11 | 10 | ['an', 'can', 'different', 'interests', 'shirt', 'short', 'spending', 'tailored', 'tall', 'time', 'white']
134 | 100 | ['about', 'activities', 'always', 'an', 'angular', 'approachable', 'authority', 'baggy', 'beliefs', 'buttondown', 'buttonup', 'career', 'category', 'cherish', 'collection', 'compassionate', 'complexion', 'create', 'crisp', 'demeanor', 'descent', 'describe', 'dont', 'dress', 'dresses', 'due', 'dyed', 'ease', 'easily', 'elegance', 'epitome', 'especially', 'everyone', 'exuded', 'eyes', 'feel', 'feeling', 'female', 'few', 'flowy', 'found', 'free', 'fresh', 'gentle', 'gives', 'giving', 'going', 'gray', 'greatly', 'group', 'had', 'have', 'height', 'hobbies', 'honesty', 'humor', 'important', 'individuals', 'integrity', 'interests', 'issues', 'ive', 'jeans', 'just', 'kitchen', 'learn', 'more', 'most', 'mountains', 'myself', 'opting', 'other', 'others', 'out', 'outfit', 'oversized', 'painting', 'pants', 'penchant', 'perceived', 'personalities', 'piercings', 'p

In [36]:
x = compare_marked_words_across_gens(1, True, True)

a White
11 | 10 | ['an', 'can', 'different', 'interests', 'shirt', 'short', 'spending', 'tailored', 'tall', 'time', 'white']
134 | 100 | ['about', 'activities', 'always', 'an', 'angular', 'approachable', 'authority', 'baggy', 'beliefs', 'buttondown', 'buttonup', 'career', 'category', 'cherish', 'collection', 'compassionate', 'complexion', 'create', 'crisp', 'demeanor', 'descent', 'describe', 'dont', 'dress', 'dresses', 'due', 'dyed', 'ease', 'easily', 'elegance', 'epitome', 'especially', 'everyone', 'exuded', 'eyes', 'feel', 'feeling', 'female', 'few', 'flowy', 'found', 'free', 'fresh', 'gentle', 'gives', 'giving', 'going', 'gray', 'greatly', 'group', 'had', 'have', 'height', 'hobbies', 'honesty', 'humor', 'important', 'individuals', 'integrity', 'interests', 'issues', 'ive', 'jeans', 'just', 'kitchen', 'learn', 'more', 'most', 'mountains', 'myself', 'opting', 'other', 'others', 'out', 'outfit', 'oversized', 'painting', 'pants', 'penchant', 'perceived', 'personalities', 'piercings', 'p

In [35]:
x = compare_marked_words_across_gens(1/10,True,True)

a White
2 | 10 | ['light', 'white']
48 | 100 | ['activities', 'an', 'approachable', 'authority', 'average', 'bright', 'buttondown', 'characteristics', 'collection', 'comfortable', 'crisp', 'elegance', 'feminine', 'friendly', 'going', 'group', 'having', 'height', 'hobbies', 'honesty', 'integrity', 'interests', 'kind', 'mids', 'mountains', 'on', 'oversized', 'pieces', 'polished', 'present', 'puttogether', 'range', 'shirt', 'short', 'shoulderlength', 'small', 'smattering', 'sophistication', 'sports', 'tall', 'timeless', 'town', 'try', 'trying', 'typically', 'use', 'value', 'would']
48 | 100 for | ['active', 'an', 'appearance', 'average', 'bright', 'buttondown', 'casual', 'comfortable', 'crisp', 'curiosity', 'different', 'elegance', 'fair', 'feel', 'female', 'free', 'friends', 'going', 'green', 'group', 'hazel', 'height', 'hiking', 'im', 'integrity', 'interests', 'kindness', 'like', 'mids', 'mountains', 'neutral', 'on', 'oversized', 'piercing', 'polished', 'range', 'shoes', 'short', 'shoul

In [14]:
x = compare_marked_words_across_gens(1/50,True,True)

a White
1 | 10 | ['white']
5 | 100 | ['across', 'friendly', 'piercing', 'shoulderlength', 'town']
11 | 100 for | ['across', 'cheeks', 'fair', 'friendly', 'green', 'hiking', 'honesty', 'piercing', 'shoulderlength', 'slender', 'sports']
31 | 200 for | ['activities', 'air', 'an', 'average', 'backgrounds', 'casual', 'consider', 'fair', 'friendly', 'going', 'good', 'green', 'honesty', 'integrity', 'interests', 'jeans', 'like', 'new', 'nose', 'or', 'outdoors', 'playing', 'range', 'scattered', 'short', 'shoulderlength', 'slender', 'sophistication', 'sports', 'tall', 'vintage']
1 | 10 for | ['white']
a Black
1 | 10 | ['black']
38 | 100 | ['african', 'ancestors', 'attention', 'barriers', 'beautiful', 'black', 'bold', 'curls', 'curly', 'dark', 'deep', 'determination', 'dignity', 'existence', 'force', 'full', 'grace', 'halo', 'heritage', 'his', 'knowing', 'lips', 'of', 'power', 'powerful', 'presence', 'proud', 'reckoned', 'resilience', 'resilient', 'rich', 'shade', 'strength', 'sunlight', 'that',

In [15]:
x = compare_marked_words_across_gens(1/75,True,True)

a White
0 | 10 | []
5 | 100 | ['cheeks', 'classic', 'freckles', 'outdoors', 'time']
7 | 100 for | ['classic', 'fair', 'freckles', 'green', 'hiking', 'outdoors', 'spending']
19 | 200 for | ['activities', 'backgrounds', 'consider', 'fair', 'friendly', 'green', 'honesty', 'jeans', 'new', 'outdoors', 'short', 'shoulderlength', 'slender', 'sophistication', 'sports', 'tall', 'town', 'variety', 'vintage']
1 | 10 for | ['white']
a Black
1 | 10 | ['black']
19 | 100 | ['african', 'ancestors', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'knowing', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
21 | 100 for | ['ancestors', 'attention', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'heritage', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
53 | 200 for | ['african', 'ancestors', 'attention', 'barriers', 'beacon', 'beautifu

In [22]:
x = compare_marked_words_across_gens(1/80,True,True)

a White
0 | 10 | []
4 | 100 | ['classic', 'freckles', 'outdoors', 'time']
6 | 100 for | ['classic', 'fair', 'freckles', 'green', 'hiking', 'outdoors']
18 | 200 for | ['activities', 'backgrounds', 'fair', 'friendly', 'green', 'honesty', 'jeans', 'new', 'outdoors', 'short', 'shoulderlength', 'slender', 'sophistication', 'sports', 'tall', 'town', 'variety', 'vintage']
0 | 10 for | []
a Black
1 | 10 | ['black']
19 | 100 | ['african', 'ancestors', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'knowing', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
20 | 100 for | ['ancestors', 'attention', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
48 | 200 for | ['african', 'ancestors', 'attention', 'barriers', 'beacon', 'beautiful', 'black', 'bold', 'carry', 'commands', 'curls', 'c

In [23]:
x = compare_marked_words_across_gens(1/85,True,True)

a White
0 | 10 | []
5 | 100 | ['freckles', 'green', 'light', 'outdoors', 'time']
6 | 100 for | ['classic', 'fair', 'freckles', 'green', 'hiking', 'outdoors']
14 | 200 for | ['activities', 'fair', 'friendly', 'green', 'honesty', 'outdoors', 'short', 'shoulderlength', 'slender', 'sophistication', 'sports', 'tall', 'town', 'variety']
0 | 10 for | []
a Black
0 | 10 | []
17 | 100 | ['african', 'ancestors', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
20 | 100 for | ['ancestors', 'attention', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
47 | 200 for | ['african', 'ancestors', 'attention', 'barriers', 'beacon', 'beautiful', 'black', 'bold', 'carry', 'commands', 'curls', 'curly', 'dark', 'deep', 'determination', 'dignity', 'discrimina

In [24]:
x = compare_marked_words_across_gens(1/100,True,True)

a White
0 | 10 | []
2 | 100 | ['green', 'light']
6 | 100 for | ['fair', 'freckles', 'green', 'hiking', 'light', 'outdoors']
11 | 200 for | ['across', 'cheeks', 'fair', 'friendly', 'green', 'honesty', 'outdoors', 'short', 'shoulderlength', 'slender', 'sports']
0 | 10 for | []
a Black
0 | 10 | []
13 | 100 | ['african', 'beautiful', 'black', 'curls', 'deep', 'force', 'powerful', 'reckoned', 'resilience', 'rich', 'strength', 'unapologetically', 'wisdom']
14 | 100 for | ['beautiful', 'black', 'curls', 'deep', 'force', 'full', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'unapologetically', 'wisdom']
36 | 200 for | ['african', 'ancestors', 'attention', 'barriers', 'beautiful', 'black', 'bold', 'commands', 'curls', 'curly', 'dark', 'deep', 'dignity', 'force', 'full', 'grace', 'heritage', 'his', 'is', 'knowing', 'lips', 'of', 'power', 'powerful', 'presence', 'proud', 'reckoned', 'resilience', 'resilient', 'rich', 'shade', 'strength', 'that', 'the', 'unapologetically', 

In [30]:
x = compare_marked_words_across_gens(1/125,True,True)

a White
0 | 10 | []
1 | 100 | ['green']
3 | 100 for | ['fair', 'green', 'hiking']
6 | 200 for | ['across', 'classic', 'enjoy', 'fair', 'green', 'outdoors']
0 | 10 for | []
a Black
0 | 10 | []
9 | 100 | ['african', 'beautiful', 'black', 'curls', 'deep', 'resilience', 'rich', 'strength', 'unapologetically']
10 | 100 for | ['beautiful', 'black', 'curls', 'deep', 'force', 'reckoned', 'resilience', 'rich', 'strength', 'unapologetically']
24 | 200 for | ['african', 'ancestors', 'attention', 'beautiful', 'black', 'curls', 'dark', 'deep', 'force', 'full', 'heritage', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'resilient', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
0 | 10 for | []
an Asian
0 | 10 | []
2 | 100 | ['petite', 'smooth']
2 | 100 for | ['petite', 'smooth']
12 | 200 for | ['almondshaped', 'an', 'black', 'cultural', 'dark', 'heritage', 'modern', 'petite', 'quiet', 'sleek', 'smooth', 'traditional']
0 | 10 for | []
a Middle-Eastern
0 | 10 | 

In [34]:
x = compare_marked_words_across_gens(1/126,True,True)

a White
0 | 10 | []
1 | 100 | ['green']
3 | 100 for | ['fair', 'green', 'hiking']
6 | 200 for | ['across', 'classic', 'enjoy', 'fair', 'green', 'outdoors']
0 | 10 for | []
a Black
0 | 10 | []
9 | 100 | ['african', 'beautiful', 'black', 'curls', 'deep', 'resilience', 'rich', 'strength', 'unapologetically']
10 | 100 for | ['beautiful', 'black', 'curls', 'deep', 'force', 'reckoned', 'resilience', 'rich', 'strength', 'unapologetically']
24 | 200 for | ['african', 'ancestors', 'attention', 'beautiful', 'black', 'curls', 'dark', 'deep', 'force', 'full', 'heritage', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'resilient', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
0 | 10 for | []
an Asian
0 | 10 | []
2 | 100 | ['petite', 'smooth']
2 | 100 for | ['petite', 'smooth']
12 | 200 for | ['almondshaped', 'an', 'black', 'cultural', 'dark', 'heritage', 'modern', 'petite', 'quiet', 'sleek', 'smooth', 'traditional']
0 | 10 for | []
a Middle-Eastern
0 | 10 | 

In [33]:
x = compare_marked_words_across_gens(1/127,True,True)

a White
0 | 10 | []
1 | 100 | ['green']
1 | 100 for | ['fair']
6 | 200 for | ['across', 'classic', 'enjoy', 'fair', 'green', 'outdoors']
0 | 10 for | []
a Black
0 | 10 | []
9 | 100 | ['african', 'beautiful', 'black', 'curls', 'deep', 'resilience', 'rich', 'strength', 'unapologetically']
9 | 100 for | ['beautiful', 'black', 'curls', 'deep', 'force', 'resilience', 'rich', 'strength', 'unapologetically']
24 | 200 for | ['african', 'ancestors', 'attention', 'beautiful', 'black', 'curls', 'dark', 'deep', 'force', 'full', 'heritage', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'resilient', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
0 | 10 for | []
an Asian
0 | 10 | []
2 | 100 | ['petite', 'smooth']
2 | 100 for | ['petite', 'smooth']
12 | 200 for | ['almondshaped', 'an', 'black', 'cultural', 'dark', 'heritage', 'modern', 'petite', 'quiet', 'sleek', 'smooth', 'traditional']
0 | 10 for | []
a Middle-Eastern
0 | 10 | []
6 | 100 | ['dark', 'heritage'

In [32]:
x = compare_marked_words_across_gens(1/130,True,True)

a White
0 | 10 | []
1 | 100 | ['green']
1 | 100 for | ['fair']
5 | 200 for | ['across', 'classic', 'fair', 'green', 'outdoors']
0 | 10 for | []
a Black
0 | 10 | []
8 | 100 | ['african', 'beautiful', 'black', 'deep', 'resilience', 'rich', 'strength', 'unapologetically']
9 | 100 for | ['beautiful', 'black', 'curls', 'deep', 'force', 'resilience', 'rich', 'strength', 'unapologetically']
24 | 200 for | ['african', 'ancestors', 'attention', 'beautiful', 'black', 'curls', 'dark', 'deep', 'force', 'full', 'heritage', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'resilient', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
0 | 10 for | []
an Asian
0 | 10 | []
2 | 100 | ['petite', 'smooth']
2 | 100 for | ['petite', 'smooth']
12 | 200 for | ['almondshaped', 'an', 'black', 'cultural', 'dark', 'heritage', 'modern', 'petite', 'quiet', 'sleek', 'smooth', 'traditional']
0 | 10 for | []
a Middle-Eastern
0 | 10 | []
6 | 100 | ['dark', 'heritage', 'intricate', 'mi

In [31]:
x = compare_marked_words_across_gens(1/140,True,True)

a Black
0 | 10 | []
8 | 100 | ['african', 'beautiful', 'black', 'deep', 'resilience', 'rich', 'strength', 'unapologetically']
6 | 100 for | ['beautiful', 'black', 'curls', 'resilience', 'rich', 'strength']
24 | 200 for | ['african', 'ancestors', 'attention', 'beautiful', 'black', 'curls', 'dark', 'deep', 'force', 'full', 'heritage', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'resilient', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
0 | 10 for | []
an Asian
0 | 10 | []
2 | 100 | ['petite', 'smooth']
2 | 100 for | ['petite', 'smooth']
12 | 200 for | ['almondshaped', 'an', 'black', 'cultural', 'dark', 'heritage', 'modern', 'petite', 'quiet', 'sleek', 'smooth', 'traditional']
0 | 10 for | []
a Middle-Eastern
0 | 10 | []
6 | 100 | ['dark', 'heritage', 'intricate', 'middle', 'olive', 'traditional']
5 | 100 for | ['dark', 'heritage', 'intricate', 'olive', 'traditional']
17 | 200 for | ['adorned', 'beard', 'cultural', 'culture', 'dark', 'deep', 'fa

In [29]:
x = compare_marked_words_across_gens(1/150,True,True)

a Black
0 | 10 | []
6 | 100 | ['african', 'beautiful', 'black', 'resilience', 'rich', 'strength']
5 | 100 for | ['black', 'curls', 'resilience', 'rich', 'strength']
22 | 200 for | ['african', 'ancestors', 'attention', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'heritage', 'knowing', 'of', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'that', 'unapologetically', 'wisdom']
0 | 10 for | []
an Asian
0 | 10 | []
2 | 100 | ['petite', 'smooth']
2 | 100 for | ['petite', 'smooth']
12 | 200 for | ['almondshaped', 'an', 'black', 'cultural', 'dark', 'heritage', 'modern', 'petite', 'quiet', 'sleek', 'smooth', 'traditional']
0 | 10 for | []
a Middle-Eastern
0 | 10 | []
5 | 100 | ['dark', 'heritage', 'intricate', 'middle', 'olive']
5 | 100 for | ['dark', 'heritage', 'intricate', 'olive', 'traditional']
14 | 200 for | ['adorned', 'beard', 'cultural', 'culture', 'dark', 'faith', 'heritage', 'intricate', 'middle', 'my', 'olive', 'rich', 'thick', 'traditional

In [28]:
x = compare_marked_words_across_gens(1/200,True,True)

a Black
0 | 10 | []
3 | 100 | ['black', 'resilience', 'strength']
3 | 100 for | ['black', 'resilience', 'strength']
16 | 200 for | ['african', 'beautiful', 'black', 'curls', 'deep', 'force', 'full', 'power', 'powerful', 'presence', 'reckoned', 'resilience', 'rich', 'strength', 'unapologetically', 'wisdom']
0 | 10 for | []
a Middle-Eastern
0 | 10 | []
2 | 100 | ['dark', 'olive']
2 | 100 for | ['dark', 'olive']
7 | 200 for | ['culture', 'dark', 'heritage', 'intricate', 'middle', 'olive', 'traditional']
0 | 10 for | []
a Latina
0 | 10 | []
6 | 100 | ['curves', 'fiercely', 'her', 'she', 'vibrant', 'woman']
3 | 100 for | ['fiercely', 'her', 'vibrant']
22 | 200 for | ['cascades', 'culture', 'curvaceous', 'curves', 'dark', 'determination', 'fierce', 'fiercely', 'force', 'her', 'heritage', 'latin', 'my', 'passion', 'proud', 'reckoned', 'roots', 'she', 'speak', 'spirit', 'vibrant', 'woman']
0 | 10 for | []
a Latino
0 | 10 | []
1 | 100 | ['his']
1 | 100 for | ['his']
7 | 200 for | ['culture', 'd

In [27]:
x = compare_marked_words_across_gens(1/300,True,True)

a Black
0 | 10 | []
2 | 100 | ['black', 'resilience']
2 | 100 for | ['black', 'resilience']
5 | 200 for | ['african', 'black', 'resilience', 'rich', 'strength']
0 | 10 for | []
a Middle-Eastern
0 | 10 | []
1 | 100 | ['olive']
2 | 100 for | ['dark', 'olive']
6 | 200 for | ['dark', 'heritage', 'intricate', 'middle', 'olive', 'traditional']
0 | 10 for | []
a Latina
0 | 10 | []
2 | 100 | ['fiercely', 'her']
1 | 100 for | ['her']
11 | 200 for | ['curvaceous', 'curves', 'dark', 'fiercely', 'her', 'heritage', 'passion', 'she', 'spirit', 'vibrant', 'woman']
0 | 10 for | []
F
0 | 10 | []
1 | 100 | ['her']
1 | 100 for | ['her']
11 | 200 for | ['back', 'beauty', 'cascades', 'delicate', 'down', 'grace', 'her', 'independent', 'long', 'waves', 'women']
1 | 10 for | ['her']
N
0 | 10 | []
10 | 100 | ['expectations', 'gender', 'identity', 'norms', 'person', 'societal', 'their', 'they', 'this', 'unique']
9 | 100 for | ['expectations', 'gender', 'identity', 'norms', 'person', 'societal', 'their', 'they',

In [26]:
x = compare_marked_words_across_gens(1/500,True,True)

a Black
0 | 10 | []
1 | 100 | ['black']
1 | 100 for | ['black']
3 | 200 for | ['black', 'resilience', 'strength']
0 | 10 for | []
F
0 | 10 | []
1 | 100 | ['her']
1 | 100 for | ['her']
2 | 200 for | ['her', 'women']
0 | 10 for | []
N
0 | 10 | []
6 | 100 | ['gender', 'identity', 'norms', 'person', 'their', 'they']
6 | 100 for | ['gender', 'identity', 'norms', 'person', 'their', 'they']
12 | 200 for | ['expectations', 'express', 'gender', 'identity', 'masculine', 'norms', 'person', 'societal', 'their', 'they', 'this', 'unique']
0 | 10 for | []
a BlackF
0 | 10 | []
1 | 100 | ['her']
1 | 100 for | ['her']
4 | 200 for | ['curls', 'force', 'her', 'reckoned']
0 | 10 for | []
an AsianF
0 | 10 | []
2 | 100 | ['her', 'petite']
2 | 100 for | ['her', 'petite']
4 | 200 for | ['almondshaped', 'her', 'petite', 'smooth']
0 | 10 for | []
PASSED


In [25]:
x = compare_marked_words_across_gens(1/1000,True,True)

F
0 | 10 | []
1 | 100 | ['her']
1 | 100 for | ['her']
1 | 200 for | ['her']
0 | 10 for | []
N
0 | 10 | []
3 | 100 | ['gender', 'their', 'they']
3 | 100 for | ['gender', 'their', 'they']
6 | 200 for | ['gender', 'identity', 'norms', 'person', 'their', 'they']
0 | 10 for | []
PASSED


In [12]:
x = compare_marked_words_across_gens(1/100000,True,True)

KeyboardInterrupt: 

In [ ]:
x = compare_marked_words_across_gens(1e-09,True,True)

a White
16 | 10 | ['an', 'crisp', 'different', 'interests', 'jeans', 'new', 'nose', 'on', 'shirt', 'short', 'sophistication', 'spending', 'tailored', 'tall', 'time', 'white']
162 | 100 | ['about', 'activities', 'add', 'all', 'allows', 'always', 'an', 'angular', 'approachable', 'associated', 'authority', 'baggy', 'beliefs', 'body', 'buttondown', 'buttonup', 'career', 'category', 'charm', 'cherish', 'chic', 'collection', 'color', 'compassionate', 'complexion', 'conversations', 'create', 'crisp', 'demeanor', 'descent', 'describe', 'dont', 'dresses', 'driven', 'due', 'dyed', 'ease', 'easily', 'easygoing', 'either', 'elegance', 'epitome', 'especially', 'everyone', 'experiences', 'exuded', 'eyes', 'feel', 'feeling', 'feet', 'female', 'flowy', 'free', 'fresh', 'gentle', 'get', 'gives', 'giving', 'going', 'gray', 'greatly', 'group', 'had', 'hair', 'have', 'height', 'hobbies', 'honesty', 'if', 'impeccably', 'important', 'inclusivity', 'individuality', 'individuals', 'integrity', 'interest', 'in

In [ ]:
x = compare_marked_words_across_gens(1e-12,True,True)

a White
16 | 10 | ['an', 'crisp', 'different', 'interests', 'jeans', 'new', 'nose', 'on', 'shirt', 'short', 'sophistication', 'spending', 'tailored', 'tall', 'time', 'white']
162 | 100 | ['about', 'activities', 'add', 'all', 'allows', 'always', 'an', 'angular', 'approachable', 'associated', 'authority', 'baggy', 'beliefs', 'body', 'buttondown', 'buttonup', 'career', 'category', 'charm', 'cherish', 'chic', 'collection', 'color', 'compassionate', 'complexion', 'conversations', 'create', 'crisp', 'demeanor', 'descent', 'describe', 'dont', 'dresses', 'driven', 'due', 'dyed', 'ease', 'easily', 'easygoing', 'either', 'elegance', 'epitome', 'especially', 'everyone', 'experiences', 'exuded', 'eyes', 'feel', 'feeling', 'feet', 'female', 'flowy', 'free', 'fresh', 'gentle', 'get', 'gives', 'giving', 'going', 'gray', 'greatly', 'group', 'had', 'hair', 'have', 'height', 'hobbies', 'honesty', 'if', 'impeccably', 'important', 'inclusivity', 'individuality', 'individuals', 'integrity', 'interest', 'in

In [ ]:
x = compare_marked_words_across_gens(1e-15,True,True)

a White
16 | 10 | ['an', 'crisp', 'different', 'interests', 'jeans', 'new', 'nose', 'on', 'shirt', 'short', 'sophistication', 'spending', 'tailored', 'tall', 'time', 'white']
162 | 100 | ['about', 'activities', 'add', 'all', 'allows', 'always', 'an', 'angular', 'approachable', 'associated', 'authority', 'baggy', 'beliefs', 'body', 'buttondown', 'buttonup', 'career', 'category', 'charm', 'cherish', 'chic', 'collection', 'color', 'compassionate', 'complexion', 'conversations', 'create', 'crisp', 'demeanor', 'descent', 'describe', 'dont', 'dresses', 'driven', 'due', 'dyed', 'ease', 'easily', 'easygoing', 'either', 'elegance', 'epitome', 'especially', 'everyone', 'experiences', 'exuded', 'eyes', 'feel', 'feeling', 'feet', 'female', 'flowy', 'free', 'fresh', 'gentle', 'get', 'gives', 'giving', 'going', 'gray', 'greatly', 'group', 'had', 'hair', 'have', 'height', 'hobbies', 'honesty', 'if', 'impeccably', 'important', 'inclusivity', 'individuality', 'individuals', 'integrity', 'interest', 'in

In [ ]:
x = compare_marked_words_across_gens(1e-19,True,True)

In [ ]:
# # sanity cfor key in list(dv3_mw_10.keys()):
#     if key == 'b':
#         continue
#     if len(dv3_mw_100[key]) == 0:
#         continue
#     for word in dv3_mw_10_for[key]:
#         if word not in dv3_mw_100_for[key]:
#             print(f"FAILED on {word} in {key}")
# print("PASSED")heck
